# Data aggregation: from daily to custom frequencies
### Rodrigo Leo
---

This script aggregates daily data to fit custom frequencies.

In [35]:
# Import the required libraries
import numpy as np
import pandas as pd

In [36]:
# Load data
data = pd.read_excel('databases/data_usext.xlsx', engine = 'openpyxl')

# Set the type of column 'date' to datetime
data['date'] = pd.to_datetime(data['date'])

# Set column 'date' as index
data = data.set_index('date')

# Show Data Frame
data

,observation,sp500_price,tbill,rpc_consumption,price
date,,,,,
1954-01-04,1,24.950001,1.33,27.716667,26.944839
1954-01-05,2,25.100000,1.28,27.716667,26.946452
1954-01-06,3,25.139999,1.28,27.716667,26.948065
1954-01-07,4,25.059999,1.31,27.716667,26.949677
1954-01-08,5,24.930000,1.31,27.716667,26.951290
...,...,...,...,...,...
2020-12-24,16684,3703.060059,0.09,107.067361,262.045300
2020-12-28,16685,3735.360107,0.11,107.067361,262.129700
2020-12-29,16686,3727.040039,0.10,107.067361,262.150800


The frequencies to be obtained are passed to the script in a dictionary called `freq`, where the keys are `string`s that represent descriptive names of the frequency (for example, `monthly`) and the values are `int`s with the number of the corresponding observations for the period (for example, `30`).

In [37]:
freq = {
    'w': 7,
    'f': 15,
    'm': 30,
    'q': 90,
    's': 180,
    'a': 360
}

In [38]:
results = {f: None for f in freq}

for f in freq.keys():
    
    # Construct the periods
    days = freq[f]
    periods = []
    i = 0
    while True:
        lim1 = days * i + 1
        lim2 = lim1 + days - 1
        if lim2 >= max(data['observation']):
            break
        else:
            periods.append((lim1, lim2))
            i = i + 1

    # Aggregate data
    aggregate_list = []
    for period in periods:
        sample = data.query(f'observation >= {period[0]} and observation <= {period[1]}')
        aggregate = {
            'date': sample.last(offset = '1D').index.to_pydatetime()[0],
            'sp500_price': sample.last(offset = '1D')['sp500_price'].values[0],
            'tbill': np.mean(sample['tbill']),
            'rpc_consumption': np.sum(sample['rpc_consumption']),
            'price': np.mean(sample['price'])
        }
        aggregate_list.append(aggregate)
    
    # Store the aggregated data in a Data Frame
    results[f] = pd.DataFrame(aggregate_list)

In [39]:
# Show the lenghts of the results
for r in results.keys():
    print(r + ': ' + str(len(results[r])) + ' observations')

w: 2383 observations
f: 1112 observations
m: 556 observations
q: 185 observations
s: 92 observations
a: 46 observations


The results are saved in the `databases` folder as `aggregated_x.csv`, where each `x` is a key of the dictionary `freq`.

In [40]:
# Save the aggregated data as .csv files
for r in results.keys():
    results[r].to_csv(f'databases/aggregated_{r}.csv')